In [1]:
import requests
import pandas as pd
import json

In [2]:
out = requests.get('https://covidtracking.com/api/states')
df_out = pd.DataFrame(out.json())
df_out.set_index('state', drop=True, inplace=True)

In [3]:
tx_new = pd.Series(dict(
    positive=194,
    negative=5083,
    death=3,
    total=5277,
    lastUpdateEt='3/20 11:00'
))
tx_new

positive               194
negative              5083
death                    3
total                 5277
lastUpdateEt    3/20 11:00
dtype: object

In [4]:
df_out.at['TX'] = tx_new

In [5]:
df_out.to_parquet('states_today.parquet')

In [15]:
ts = requests.get('https://covidtracking.com/api/states/daily')
df_ts = pd.DataFrame(ts.json())

In [16]:
tx_new = pd.Series(dict(
    date=20200320,
    state='TX',
    positive=194,
    negative=5083,
    death=3,
    total=5277,
))
tx_new

date        20200320
state             TX
positive         194
negative        5083
death              3
total           5277
dtype: object

In [17]:
i = df_ts[(df_ts['date'] == 20200313) & (df_ts.state == 'TX')].index[0]
df_ts.at[i, 'total'] = 220
df_ts.at[i, 'death'] = 0
df_ts.at[i, 'negative'] = df_ts.at[i, 'total'] - df_ts.at[i, 'positive'] 
df_ts.loc[i]

date                       20200313
state                            TX
positive                         39
negative                        181
pending                         NaN
death                             0
total                           220
dateChecked    2020-03-13T20:00:00Z
Name: 369, dtype: object

In [18]:
df_ts = df_ts.append(tx_new, ignore_index=True)
df_ts.tail()

,date,state,positive,negative,pending,death,total,dateChecked
753,20200304,SC,0,5.0,0.0,NaN,5,2020-03-04T21:00:00Z
754,20200304,TX,1,NaN,NaN,NaN,1,2020-03-04T21:00:00Z
755,20200304,WA,39,NaN,NaN,NaN,39,2020-03-04T21:00:00Z
756,20200304,WI,1,19.0,6.0,NaN,26,2020-03-04T21:00:00Z
757,20200320,TX,194,5083.0,NaN,3.0,5277,NaN


In [19]:
df_ts.to_parquet('states_daily.parquet')